# Default Precomputations

This notebook contains the standard precomputation steps for normal motif counting settings.

Simply choosing your setting via the respective `settings` module and running the notebook cells should be enough to be ready to count on arbitrary host graphs. 

At the end of the notebook we will store all of our precomputed structures, allowing us to easily reuse them for couting different host graphs.

Completed precomputations are already included in standard distribution of pact for treelets of size 5 to 10. Those can be directly used using the Standard Counting notebook with no need for new precomputation.

In [ ]:
%reload_ext rich

import networkx as nx
from itertools import repeat
import dill
from pact.ui import default_progressbar
import multiprocess as mp
from pact.graphwrapper import GraphWrapper
from pact.hombase import *
from pact.spasmspace import SpasmSpace
from pact.balgowrapper import balgo_multitry_for_cheapest_decomp
from pact.planner import node_to_ops, node_to_ops_earlysj


from settings.treelets import *
# The exact same code also works for the `all_directed` configuration of all connected graphs with k nodes.
# The only change that is needed is to import the respective other settings module
# instead of the treelets settings module.
# from settings.all_directed import *

K = 8

# Load Plain Graphs

In [ ]:
pattern_file = raw_pattern_filename(K)
spasm_files = raw_spasm_filenames(K)
spasm_space = SpasmSpace()


for sf in spasm_files:
    with open(sf) as f:
        lines = list(map(str.rstrip, f.readlines()))
        spasm_space.add_from_g6lines(lines, SPASM_GRAPHWRAPPER_PARAMS)

patterns = dict()
with open(pattern_file) as f:  # tree8
    lines = list(map(str.rstrip, f.readlines()))
    for line in lines:
        g = GraphWrapper.from_g6str(line, **PATTERN_GRAPHWRAPPER_PARAMS)
        patterns[g.id] = g
        
print('Spasm base has', len(spasm_space), 'graphs')
print('#patterns:', len(patterns))

# Compute everything (in parallel)


In [ ]:
THREADS = 4
def _helper(H):
    return H.id, hombase_coeffs_nauty(H, spasm_space)

with default_progressbar() as progress:
    pool = mp.Pool(THREADS)
    track = progress.track(pool.imap_unordered(_helper, patterns.values()),
                           total=len(patterns))
    for hid, coeffs in track:
        patterns[hid].hombase = coeffs

# Precompute Decompositions

In [ ]:
progress = default_progressbar()
badness_acc = []
with progress:
    track = progress.track(list(enumerate(spasm_space.graphs_iter())))
    for i, G in track:
        decomp, cost = balgo_multitry_for_cheapest_decomp(G, times=40, threads=3)
        G.td = decomp
        G.td_badness = cost
        badness_acc.append(cost)

# We'd like these to be low
print('Average badness: ', sum(badness_acc)/len(badness_acc)) 
print('Max badness: ', max(badness_acc))

# Add Execution Plans to our Spasm Space

In [ ]:
with default_progressbar() as progress:
    track = progress.track(list(spasm_space.graphs_iter()))
    for G in track:
        G.plan = node_to_ops_earlysj(G.td)

# Save our created structures

In [ ]:

with open(spasm_filename(K), 'wb') as f:
    spasm_space.cleanup_for_storage()
    f.write(dill.dumps(spasm_space))

with open(patterns_filename(K), 'wb') as f:
    f.write(dill.dumps(patterns))